In [1]:
import numpy as np
import scipy
import numpy 
import matplotlib.pyplot as plt
from scipy import linalg, special, stats
from numpy import genfromtxt
import ML_support as ml
from scipy.optimize import fmin_l_bfgs_b

In [2]:
def logreg_obj_wrap(DTR, LTR, l):

    def logreg_obj(v):
        n = DTR.shape[1]
        w, b = v[0:-1], v[-1]
        w = w.reshape((len(w), 1))
        J = 0
        x = DTR
        for idx in range(n):
            if LTR[idx] == 0:
                c = 0
            else:
                c = 1

            J += (c * numpy.log1p(numpy.exp(numpy.dot(-w.T, x[:, idx]) - b)) + (1 - c) * numpy.log1p(numpy.exp(numpy.dot(w.T, x[:, idx]) + b)))

        return l/2 * ((numpy.linalg.norm(w))**2) + 1/n * J

    return logreg_obj

In [10]:
def logreg(DTR, LTR, DTE, LTE, l):
    logreg_obj = logreg_obj_wrap(DTR, LTR, l)
    v = numpy.zeros((DTR.shape[0]+1), dtype='int32')
    x, f, d = fmin_l_bfgs_b(logreg_obj, v, approx_grad=True)
    w, b = x[0:-1], x[-1]
    
    w = ml.mcol(w)
    S = numpy.zeros((DTE.shape[1]))
    for i in range(len(S)):
        S[i] = numpy.dot(w.T, DTE[:, i]) + b

    PL = numpy.zeros((len(LTE)))

    for i in range(len(S)):
        if(S[i] > 0):
            PL[i] = 1

    PL = PL == LTE
    correctPred = sum(PL)

    e = (len(LTE)-correctPred)/len(LTE)
    print(f'| lambda = {l} |{logreg_obj(x)}|   {round(e*100, 1)}%     |\n')
        

In [13]:
Data, label = ml.loadFile('../Train.txt')
(DTR, LTR), (DTE,LTE) = ml.split_db_2to1(Data, label, seed=42)
DTR_g, DTE_g = ml.gaussianize(DTR), ml.gaussianize(DTE)
DTR_z, DTE_z = ml.z_normalization(DTR), ml.z_normalization(DTE)


In [5]:
logreg(DTR_g, LTR, DTE_g, LTE, 0)

| l = 0 |[0.08436835]|   2.4%     |



In [6]:
logreg(DTR_g, LTR, DTE_g, LTE, 0.1)

| l = 0.1 |[0.16196515]|   4.3%     |



In [7]:
logreg(DTR_g, LTR, DTE_g, LTE, 0.01)

| l = 0.01 |[0.1075801]|   2.9%     |



In [8]:
logreg(DTR_g, LTR, DTE_g, LTE, 0.001)

| l = 0.001 |[0.09114929]|   2.5%     |



In [9]:
logreg(DTR_g, LTR, DTE_g, LTE, 0.00001)

| l = 1e-05 |[0.08455873]|   2.4%     |



In [11]:
logreg(DTR_g, LTR, DTE_g, LTE, 0.00000001)

| lambda = 1e-08 |[0.08436856]|   2.4%     |



In [14]:
logreg(DTR_z, LTR, DTE_z, LTE, 0)

| lambda = 0 |[0.07299565]|   2.2%     |



In [15]:
logreg(DTR_z, LTR, DTE_z, LTE, 0.1)
logreg(DTR_z, LTR, DTE_z, LTE, 0.01)
logreg(DTR_z, LTR, DTE_z, LTE, 0.001)
logreg(DTR_z, LTR, DTE_z, LTE, 0.00001)

| lambda = 0.1 |[0.14874581]|   4.1%     |

| lambda = 0.01 |[0.10062142]|   2.9%     |

| lambda = 0.001 |[0.08311847]|   2.5%     |

| lambda = 1e-05 |[0.07334328]|   2.2%     |



In [16]:
logreg(DTR_z, LTR, DTE_z, LTE, 0.0000001)

| lambda = 1e-07 |[0.07299922]|   2.2%     |

